In [2]:
#import packages
import numpy as np
import json
import scanpy as sc
from collections import OrderedDict
import scipy
import pandas as pd
import matplotlib.pyplot as plt

#spectra imports
import Spectra as spc
from Spectra import Spectra_util as spc_tl
from Spectra import K_est as kst
from Spectra import default_gene_sets


In [3]:
skin_data = sc.read_h5ad('/krummellab/data1/DSCoLab/AUTOIPI/papers/skin/data/DB3_54/harmony/sobj_01-13-25.h5ad')

In [4]:
skin_data

AnnData object with n_obs × n_vars = 261290 × 28683
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'doubletFinderScore', 'nCount_ADT', 'nFeature_ADT', 'percent.mt', 'percent.ribo', 'isotype_ctl_max', 'S.Score', 'G2M.Score', 'Phase', 'pool', 'batch', 'biospecimen', 'individual', 'subtype', 'disease_group', 'years_since_diagnosis', 'Age', 'Sex at Birth', 'sample_type', 'd_sample_type', 'location', 'cluster_label'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable'
    obsm: 'X_harmony', 'X_pca', 'X_umap'
    varm: 'HARMONY', 'PCs'